<a href="https://colab.research.google.com/github/ohmkas/fundos-de-investimento-/blob/main/An%C3%A1lise_de_investimento_em_FI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pré-requisitos

## Pacotes

In [ ]:
! pip install yfinance --upgrade --no-cache-dir
! pip install plotly==4.9.0
! pip install -U kaleido
! pip install psutil
! pip install chart-studio==1.0.0
! pip install seaborn

     |████████████████████████████████| 5.5MB 6.3MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.55-py2.py3-none-any.whl size=22616 sha256=97186c6c02e8825b174aee4777c1cd1d361ac33311ce9f97b41d92a441280bfa
  Stored in directory: /tmp/pip-ephem-wheel-cache-4jyoshwd/wheels/04/98/cc/2702a4242d60bdc14f48b4557c427ded1fe92aedf257d4565c
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
     |████████████████████████████████| 12.9MB 258kB/s 
  Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1
     |████████████████████████████████| 79.9MB 73kB/s 
     |████████████████████████████████| 81kB 3.5MB/s 


In [ ]:
import numpy as np
import pandas as pd
import plotly.figure_factory as ff
import math
import matplotlib.pyplot as plt
import pandas_datareader.data as web
import yfinance as yf 
yf.pdr_override()
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from pandas.tseries.offsets import BDay
from datetime import date, datetime
from PIL import Image 
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
import os

# Salvamento 

In [ ]:
nome_pasta = 'Rodrigo' #local onde será salvo
  
if not os.path.exists("/content/drive/My Drive/"+nome_pasta): #se não existir a pasta  
    os.mkdir("/content/drive/My Drive/"+nome_pasta) #crie a pasta 

caminho = str("/content/drive/My Drive/"+nome_pasta) #salve nessa pasta, nesse caminho

## Base de dados

In [ ]:
inicio_analise = '2020-01'

In [ ]:
def consulta_bc(codigo_bcb): #função que coleta dados do Banco Central
    url = 'http://api.bcb.gov.br/dados/serie/bcdata.sgs.{}/dados?formato=json'.format(codigo_bcb)
    df = pd.read_json(url)
    df['data'] = pd.to_datetime(df['data'], dayfirst=True)
    df.set_index('data', inplace=True)
    return (df)
 
#https://www3.bcb.gov.br/sgspub/consultarvalores/telaCvsSelecionarSeries.paint

In [ ]:
#benchmark's
cdi = consulta_bc(12).loc[inicio_analise:] #coleta o CDI - ativo livre de risco do Brasil
cdi.index = pd.to_datetime(cdi.index) #corrige bug em datas (1)

ibov = web.get_data_yahoo('^BVSP')['Adj Close'].loc[inicio_analise:] #coleta o Ibovespa - indicador renda variável do Brasil
ibov.index = pd.to_datetime(ibov.index)#idem (1)

sep = web.get_data_yahoo('^GSPC')['Adj Close'].loc[inicio_analise:] #Colet o S&P - indicador de renda variável dos EUA
sep.index = pd.to_datetime(sep.index)#idem (1)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [ ]:
#BASE DE DADOS DA CVM

cadastro = pd.read_csv('http://dados.cvm.gov.br/dados/FI/CAD/DADOS/cad_fi.csv', 
                       sep=';', encoding='ISO-8859-1',
                       index_col='CNPJ_FUNDO') #busca o arquivo de cadastro dos FI's da base da CVM


datas = pd.date_range(inicio_analise, date.today(), freq='MS')  #período analisado 
informes = pd.DataFrame() #DataFrame que terá armazenado todos os informes
 
for data in datas:
  try:
    url ='http://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_{}{:02}.csv'.format(data.year, data.month)
    informe_mensal = pd.read_csv(url, sep=';')    #coleta os dados da base de dados da CVM
  
  except: 
    print("Arquivo {} não encontrado!".format(url))    #se alterarem o URL dos dados, isso será printado
 
  informes = pd.concat([informes, informe_mensal], ignore_index=True) #acumula os informes diários


#******************************************************************************************************  


arrays = [informes['CNPJ_FUNDO'], informes['DT_COMPTC']] #serão agrupados conforme cnpj e data
tuples = list(zip(*arrays)) 
index = pd.MultiIndex.from_tuples(tuples, names=['Fundos', 'Datas']) 
informes.index = index 
informes = informes.drop(columns=['CNPJ_FUNDO', 'DT_COMPTC']) #excluí as colunas que se tornaram indices


#*********************************************************************************************************

#candidatos para investir devem respeitar os critérios seguintes:

#1 - devem possuir patrimônio líquido positivo
#2 - devem ser destinados a investidores em geral

candidatos = informes.xs(str(informes.index[-1][1]),
            level='Datas').sort_values('VL_PATRIM_LIQ',
                                       ascending=False).loc[informes.xs(str(informes.index[-1][1]),
            level='Datas')['NR_COTST'] > 1000] 
#fundos de investimento ordenados pelo patrimônio líquido positivo e pelo número de cotistas superior a mil investidores (investidores em geral)


#************************************************************ correção entre incompatibildiade do comprimento entre candidatos e cadastro


classes = [] #armazenara as classes 
cnpj_equivalente = [] 
for i in candidatos.index: #percorre os cnpj dos candidatos existentes 
  aux = (cadastro.loc[
                 candidatos.index].loc[
                 cadastro.loc[
                 candidatos.index]['SIT'] != 'CANCELADA']['CLASSE'].loc[i]) #coleta a classe do candidato correspondente 
  cnpj_equivalente.append(i) #adiciona o cnpj correspondente
  if type(aux) == type('FI'): #se estiver como tipo string, estará no formato correto 
    classes.append(aux) 
  elif type(aux) == type(3.67): #se apresentar float, estára como nan, precisa ser alterado
    classes.append('NÃO INFORMADO')
  else: 
    classes.append(aux.drop_duplicates()[0]) #se apresentar pandas, estará como duplo, pois o loc trouxe mais de um valor

candidatos = pd.concat([candidatos,pd.DataFrame(classes,index=cnpj_equivalente, columns=['CLASSE'])], axis=1)


tipos = [] #armazenara os tipos 
cnpj_equivalente = [] 
for i in candidatos.index: #percorre os cnpj dos candidatos existentes 
  aux = (cadastro.loc[
                 candidatos.index].loc[
                 cadastro.loc[
                 candidatos.index]['SIT'] != 'CANCELADA']['TP_FUNDO'].loc[i]) #coleta a classe do candidato correspondente 
  cnpj_equivalente.append(i) #adiciona o cnpj correspondente
  if type(aux) == type('FI'): #idem acima
    tipos.append(aux) 
  elif type(aux) == type(3.67): ##idem acima
    tipos.append('NÃO INFORMADO')
  else: 
    tipos.append(aux.drop_duplicates()[0]) #idem acima

candidatos = pd.concat([candidatos,pd.DataFrame(tipos,index=cnpj_equivalente, columns=['TIPOS'])], axis=1)


qualificado = [] #armazenará o tipo do investidor
cnpj_equivalente = []
for i in candidatos.index: #percorre os cnpj dos candidatos existentes 
  aux = (cadastro.loc[
                 candidatos.index].loc[
                 cadastro.loc[
                 candidatos.index]['SIT'] != 'CANCELADA']['INVEST_QUALIF'].loc[i]) #coleta a classe do candidato correspondente 
  cnpj_equivalente.append(i) #adiciona o cnpj correspondente
  if type(aux) == type('N'): #idem acima
    qualificado.append(aux)
  elif type(aux) == type(3.67): ##idem acima
    qualificado.append('NÃO INFORMADO')
  else: 
    qualificado.append(aux.drop_duplicates()[0]) #idem acima

candidatos = pd.concat([candidatos,pd.DataFrame(qualificado,index=cnpj_equivalente, columns=['QUALIFICADO'])], axis=1)

candidatos.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (14,17,18,20,22,24,27,37,38) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,VL_TOTAL,VL_QUOTA,VL_PATRIM_LIQ,CAPTC_DIA,RESG_DIA,NR_COTST,CLASSE,TIPOS,QUALIFICADO
Fundos,,,,,,,,,
04.288.966/0001-27,9.822136e+10,3.716436,9.821452e+10,1.435921e+09,1.886327e+09,169098,Fundo de Renda Fixa,FI,N
07.214.377/0001-92,4.030132e+10,4.484043,4.030050e+10,1.406636e+09,3.772111e+08,1324,Fundo de Renda Fixa,FI,N
00.071.477/0001-68,3.256353e+10,10.030132,3.256122e+10,1.883809e+09,2.004002e+09,302645,Fundo de Renda Fixa,FI,N
05.102.500/0001-58,2.905020e+10,4.648832,2.904818e+10,1.141322e+09,1.203020e+09,1150388,Fundo de Renda Fixa,FI,N
00.834.074/0001-23,2.667689e+10,5.985337,2.666804e+10,6.247122e+08,6.002436e+08,15824,Fundo de Renda Fixa,FI,N


# Seleção dos fundos



In [ ]:
candidatos['CLASSE'].drop_duplicates().to_list() #opções que podemos escolher

['Fundo de Renda Fixa',
 'Fundo Multimercado',
 'Fundo de Ações',
 'NÃO INFORMADO',
 'Fundo Cambial']

In [ ]:
classe_desejada = 'Fundo de Ações' #inserir aqui a opção desejada
cnpjs = candidatos.loc[candidatos['QUALIFICADO'] == 'N'].loc[candidatos.loc[candidatos['QUALIFICADO'] == 'N']['CLASSE'] == classe_desejada].index.to_list() #faz uma lista das opções da classe desejada 

compara_com = 'Ibovespa' #benchmark desejado de ser comparado
capital_investido = 1000 #capital inicial
fundos_vl_quota = pd.DataFrame()
for i in cnpjs:
  fundos_vl_quota[i] = informes.loc[i]['VL_QUOTA'].copy() #busca o valor da cota dos cnpj solicitados
fundos_vl_quota.index = pd.to_datetime(fundos_vl_quota.index)

valor_diario = fundos_vl_quota.pct_change() #informa a variação diária dos ativos
valor_acumulado = (1+valor_diario).cumprod() #informa a rentabilidade acumulada de cada ativo
drawdowns = (1 - valor_acumulado.div(valor_acumulado.cummax()))*-1 #gera o drawdown de cada fundo

criterios = pd.DataFrame()
criterios['Média Diária'] = valor_diario.mean() #calcula a media
criterios['D.P Diário' ] = valor_diario.std() #calcula o desvio padrao
criterios['Rentabilidade Acum'] = valor_acumulado.iloc[-1] #busca o retorno acumulado
criterios['Drawdown Máximo'] = drawdowns.min() #drawdown máximo do período
criterios['Rentabilidade Máxima'] = valor_acumulado.max() #rentabilidade máxima do período

ret_tri = []
dp_tri = []
nenhum_neg = []
for i in valor_diario.columns:
  aux = ((valor_diario[i]+1).rolling(66).apply(np.prod)-1).mean() #calcula a media da rent.
  aux1 = aux = ((valor_diario[i]+1).rolling(66).apply(np.prod)-1).std() #calcula o dp do trimestre
  ret_tri.append(aux)
  dp_tri.append(aux1)
  aux2 = ((valor_diario[i]+1).rolling(66).apply(np.prod)-1).min()
  if aux2 >0:
    nenhum_neg.append('+')
  else: 
    nenhum_neg.append('+-')

criterios['Rentabilidade Trim.'] = ret_tri #concentração dos retornos trimestrais
criterios['D.P Tri'] = dp_tri #desvios padros do trimestre
criterios['Violino'] = nenhum_neg

filtro_criterio = criterios.loc[criterios['Violino'] == '+'] #com rent. maxima maior q a mediia
print(len(filtro_criterio), '0º filtro')
filtro_criterio = filtro_criterio.loc[filtro_criterio['Média Diária'] >0] #somente fundos com a média diária positiva
print(len(filtro_criterio), '1º filtro')
filtro_criterio = filtro_criterio.loc[filtro_criterio['Rentabilidade Acum'] > 0] #com rentabilidade atual positiva
print(len(filtro_criterio), '2º filtro')
filtro_criterio = filtro_criterio.loc[filtro_criterio['D.P Diário'] <= filtro_criterio['D.P Diário'].mean()] #com baixa volatilidade diaria
print(len(filtro_criterio), '3º filtro')
filtro_criterio = filtro_criterio.loc[filtro_criterio['Drawdown Máximo'] <= filtro_criterio['Drawdown Máximo'].mean()] #com quedas pequenas
print(len(filtro_criterio), '4º filtro')
filtro_criterio = filtro_criterio.loc[filtro_criterio['Rentabilidade Máxima'] >= filtro_criterio['Rentabilidade Máxima'].mean()] #com rent. maxima maior q a mediia
print(len(filtro_criterio), '5º filtro')
filtro_criterio = filtro_criterio.sort_values(by='Rentabilidade Trim.', ascending=False).iloc[:20] #as 20 maiores rentabilidades
print(len(filtro_criterio), '6º filtro')
filtro_criterio = filtro_criterio.sort_values(by='D.P Tri', ascending=True).iloc[:10] #ordena pelos menores d.p do retorno trimestral, calda curta.
print(len(filtro_criterio), '7º filtro')
filtro_criterio #candidatos filtrados 

17 0º filtro
17 1º filtro
17 2º filtro
9 3º filtro
4 4º filtro
1 5º filtro
1 6º filtro
1 7º filtro


,Média Diária,D.P Diário,Rentabilidade Acum,Drawdown Máximo,Rentabilidade Máxima,Rentabilidade Trim.,D.P Tri,Violino
37.306.536/0001-40,0.00148,0.010515,1.286644,-0.09753,1.286644,0.033429,0.033429,+


In [ ]:
pd.DataFrame(cadastro.loc[filtro_criterio.index]['GESTOR']).dropna() #escolher os fi desejados, caso todos, utilizar o index já inserido 

,GESTOR
CNPJ_FUNDO,
37.306.536/0001-40,ITAU UNIBANCO S.A.


caso desejar checar fundos específicos no programa, substitua abaixo

In [360]:
fundos_cnpj = filtro_criterio.index #<<<<<<<<--- inserir sob forma de lista os CNPJ's dos fundos que quer analisar


fundos_vl_total       = pd.DataFrame() #valor total do fundo 
fundos_vl_quota       = pd.DataFrame() #valor da quota do fundo
fundos_vl_pt_liq      = pd.DataFrame() #valor do patrimônio líquido do fundo
fundos_vl_captado     = pd.DataFrame() #valor captado por dia
fundos_vl_resgatado   = pd.DataFrame() #valor resgatado por dia
fundos_vl_cotistas    = pd.DataFrame() #numero de cotistas no dia
 
for i in fundos_cnpj:
  fundos_vl_total[i] = informes.loc[i]['VL_TOTAL'].copy() 
  fundos_vl_quota[i] = informes.loc[i]['VL_QUOTA'].copy()
  fundos_vl_pt_liq[i] = informes.loc[i]['VL_PATRIM_LIQ'].copy()
  fundos_vl_captado[i] = informes.loc[i]['CAPTC_DIA'].copy()
  fundos_vl_resgatado[i] = informes.loc[i]['RESG_DIA'].copy()
  fundos_vl_cotistas[i] = informes.loc[i]['NR_COTST'].copy()


fundos_vl_total = fundos_vl_total.pct_change() #variação diária %
fundos_vl_quota = fundos_vl_quota.pct_change() #variação diária %
fundos_vl_pt_liq = fundos_vl_pt_liq.pct_change() #variação diária %
fundos_vl_captado = fundos_vl_captado.diff() #variação diária unid
fundos_vl_resgatado = fundos_vl_resgatado.diff() #variação diária unid
fundos_vl_cotistas = fundos_vl_cotistas.diff() #variação diária unid

#************************************corrige data para formato datatime
fundos_vl_quota.index = pd.to_datetime(fundos_vl_quota.index) 
fundos_vl_total.index = pd.to_datetime(fundos_vl_total.index)
fundos_vl_pt_liq.index = pd.to_datetime(fundos_vl_pt_liq.index)
fundos_vl_captado.index = pd.to_datetime(fundos_vl_captado.index)
fundos_vl_resgatado.index = pd.to_datetime(fundos_vl_resgatado.index)
fundos_vl_cotistas.index = pd.to_datetime(fundos_vl_cotistas.index)

#*****************************************************************8

fundos_vl_quota['CDI'] = cdi/100 #já vem com var. diaria em %
fundos_vl_quota['Ibovespa'] = ibov.pct_change() #var. dia %
fundos_vl_quota['S&P'] = sep.pct_change() #var. dia %


fundos_diario = {'VL_TOTAL': fundos_vl_total.dropna(),
                    'VL_QUOTA': fundos_vl_quota.dropna(),
                    'VL_PATRIM_LIQ': fundos_vl_pt_liq.dropna(),
                    'CAPTC_DIA':fundos_vl_captado.dropna(),
                    'RESG_DIA':fundos_vl_resgatado.dropna(), 
                    'NR_COTST':fundos_vl_cotistas.dropna()} #reune informações para conveniencia futura

#**********************************************************************************************


fundos_vl_total_acum = (1+fundos_diario['VL_TOTAL']).cumprod()  #acumula os retornos diários
fundos_vl_quota_acum = (1+fundos_diario['VL_QUOTA']).cumprod()
fundos_vl_pt_liq_acum = (1+fundos_diario['VL_PATRIM_LIQ']).cumprod()


fundos_vl_captado_acum = pd.DataFrame()
fundos_vl_resgatado_acum = pd.DataFrame()
fundos_vl_cotistas_acum = pd.DataFrame()

fundos_vl_total_acum = (1+fundos_diario['VL_TOTAL']).cumprod()  #acumula os retornos diários
fundos_vl_quota_acum = (1+fundos_diario['VL_QUOTA']).cumprod()
fundos_vl_pt_liq_acum = (1+fundos_diario['VL_PATRIM_LIQ']).cumprod()

fundos_vl_total_acum.iloc[0] = 1
fundos_vl_quota_acum.iloc[0] = 1
fundos_vl_pt_liq_acum.iloc[0] =1

for i in fundos_cnpj:
  fundos_vl_captado_acum[i] = informes.loc[i]['CAPTC_DIA'].copy()
  fundos_vl_resgatado_acum[i] = informes.loc[i]['RESG_DIA'].copy()
  fundos_vl_cotistas_acum[i] = informes.loc[i]['NR_COTST'].copy()



fundos_acumulado = {'VL_TOTAL': fundos_vl_total_acum.dropna(),
                    'VL_QUOTA': fundos_vl_quota_acum.dropna(),
                    'VL_PATRIM_LIQ': fundos_vl_pt_liq_acum.dropna(),
                    'CAPTC_DIA':fundos_vl_captado_acum.dropna().dropna(),
                    'RESG_DIA': fundos_vl_resgatado_acum.dropna().dropna(), 
                    'NR_COTST': fundos_vl_cotistas_acum.dropna().dropna()} #reune informações para conveniencia futura


informacoes_fundos = {'Diaria': fundos_diario,
                      'Acumulada': fundos_acumulado,
                      }

### Retornos

In [361]:
retornos = pd.DataFrame()
for i in informacoes_fundos['Diaria']['VL_QUOTA']:
  retornos[i] = ((informacoes_fundos['Diaria']['VL_QUOTA'][i]+1).rolling(66).apply(np.prod)-1)
retornos.describe().sort_values(by='mean',axis=1,ascending= False)
#do maior retorno médio mensal para o menor ->>>>>>>>

,Ibovespa,37.306.536/0001-40,S&P,CDI
count,104.000000,104.000000,104.000000,104.000000
mean,0.101679,0.095716,0.069793,0.004975
std,0.105553,0.040562,0.036263,0.000068
min,-0.097561,0.014047,-0.000401,0.004942
25%,0.003829,0.064707,0.042881,0.004942
50%,0.093010,0.087970,0.070267,0.004942
75%,0.183883,0.121866,0.094867,0.004944
max,0.320118,0.214507,0.180697,0.005196


In [ ]:
#padronização das cores nos gráficos
cores = [
                'blueviolet', 'brown', 'burlywood', 'cadetblue',
                'chartreuse', 'chocolate', 'coral', 'cornflowerblue',
                'cornsilk', 'crimson', 'cyan', 'darkblue', 'darkcyan',
                'darkgoldenrod', 'darkgray', 'darkgrey', 'darkgreen',
                'darkkhaki', 'darkmagenta', 'darkolivegreen', 'darkorange',
                'darkorchid', 'darkred', 'darksalmon', 'darkseagreen',
                'darkslateblue', 'darkslategray', 'darkslategrey',
                'darkturquoise', 'darkviolet', 'deeppink', 'deepskyblue',
                'dimgray', 'dimgrey', 'dodgerblue', 'firebrick',
                'floralwhite', 'forestgreen', 'fuchsia', 'gainsboro',
                'ghostwhite', 'gold', 'goldenrod', 'gray', 'grey', 'green',
                'greenyellow', 'honeydew', 'hotpink', 'indianred', 'indigo',
                'ivory', 'khaki', 'lavender', 'lavenderblush', 'lawngreen',
                'lemonchiffon', 'lightblue', 'lightcoral', 'lightcyan',
                'lightgoldenrodyellow', 'lightgray', 'lightgrey' ]

cores_em_uso = []
n=0
for i in informacoes_fundos['Diaria']['VL_QUOTA'].columns.to_list(): #a cada elemento analisado
  cores_em_uso.append(cores[n]) #adicionará uma cor
  n+=1
print(cores_em_uso)

['blueviolet', 'brown', 'burlywood', 'cadetblue']


## Informações gerais

In [368]:
fig = make_subplots(rows=2 ,
                    cols=len(informacoes_fundos['Diaria'].keys()),
                    subplot_titles=("VAR. VL TOTAL", "VAR. VL QUOTA", "VAR. VL PATRIM LQ", "VAR. CAPTAÇÃO", 'VAR. RESGATE', 'VAR. COTISTAS',
                                    "VL TOTAL ACUM.", "Vl QUOTA ACUM.", "VL PATRIM LQ ACUM.", "CAPTAÇÃO ACUM.", 'RESGATE ACUM.', 'COTISTAS ACUM.'))

z=1 #parametro para diferenciar diário de acumulado (t) 
for t in informacoes_fundos.keys(): #percorre os periodos
  c=0 #parametro para manter cores homogêneas
  n=1 #parametro da linha utilizada, deve iniciar em 1 pois é o mínimo para subplot
  
  if t == 'Diaria': #para ocultar as legendas, utilizei essa técnica. Apenas as legendas diárias aparecerão
    for i in informacoes_fundos[t].keys(): #percorre os dados dos informes
      if i == 'VL_QUOTA': #quando chegar no valor da cota
        aux=n  #travar n em aux, pois há cdi, ibovespa e s&p junto
        c=0 #zera o parametro c para retornar a cor inicial 
        for y in informacoes_fundos[t][i].columns: #percorrer todas as colunas da cota
          fig.add_trace(go.Scatter(y=informacoes_fundos[t][i][y], #ativo da vez
                                  x=informacoes_fundos[t][i][y].index, mode="lines", #seu indice respectivo
                                  name=y, #ativo
                                  legendgroup=y, #grupo do ativo
                                  line=dict(color=cores_em_uso[c]), #cor do ativo
                                  ),row=z,col=aux) #parameros evoluem de acordo com a informacao
          c+=1 #segue a sequencia padrao das cores

      
      
      else: #se for diferente de VL_QUOTA, ou seja, todas as outras informações disponíeis
        c=0 #zera a cor para manter homogenidade na sequencia 
        for y in informacoes_fundos[t][i].columns: #percorre a coluna do df da vez (vl_total, patrimonio_liquido, resgate, captação, etc.)
          fig.add_trace(go.Scatter(y=informacoes_fundos[t][i][y],
                                  x=informacoes_fundos[t][i][y].index, mode="lines",
                                  name=y,
                                  legendgroup=y,
                                  showlegend=False,
                                  line=dict(color=cores_em_uso[c])),row=z,col=n)
          c+=1

      n+=1 #percorre a coluna 
    z+=1 # percorre a linha
  
  else: #se nao for variação diaria, entao todas as legendas serao ocultadas
    for i in informacoes_fundos[t].keys(): #percorre os dados dos informes
      if i == 'VL_QUOTA': #quando chegar no valor da cota
        aux=n  #travar n em aux
        c=0
        for y in informacoes_fundos[t][i].columns: #percorrer todas as colunas da cota
          fig.add_trace(go.Scatter(y=informacoes_fundos[t][i][y],
                                  x=informacoes_fundos[t][i][y].index, mode="lines",
                                  name=y,
                                  legendgroup=y,
                                  showlegend=False, #oculta as legendas
                                  line=dict(color=cores_em_uso[c])),row=z,col=aux)
          c+=1

      
      
      else:
        c=0
        for y in informacoes_fundos[t][i].columns:
          fig.add_trace(go.Scatter(y=informacoes_fundos[t][i][y],
                                  x=informacoes_fundos[t][i][y].index, mode="lines",
                                  name=y,
                                  legendgroup=y,
                                  showlegend=False, #oculta as legendas
                                  line=dict(color=cores_em_uso[c]),
                                   ),row=z,col=n)
          c+=1

      n+=1
    z+=1
            
fig.update_layout(
    height=600, 
    width=1300,
    title = 'Informações Gerais',
    template="plotly_white",
    legend_title_text='Comparação entre:',
    legend_orientation="h",
    legend=dict(x=0, y=-.2),
    font=dict(
        family="Times New Roman",
        size=14,
        color="DimGray"
    )
)


fig.show()
fig.write_image(caminho+"/informações_gerais.png")

## Porcentagem dos retornos mensais positivos

In [ ]:
retornos_positivos = pd.DataFrame() #armazena retornos positivos
retornos_negativos = pd.DataFrame() #armazena retornos negavitos
for i in informacoes_fundos['Diaria']['VL_QUOTA'].columns: #percorre os ativos existentes
  df = ((informacoes_fundos['Diaria']['VL_QUOTA'][i]+1).rolling(66).apply(np.prod)-1).dropna() #calcula o retorno trimestral
  df = pd.DataFrame(df) #transforma em df 
  df_positivo = df.loc[df[i] >0][i] #somente valores positvos 
  df_negativo = df.loc[df[i] <=0][i] #somente valores negativos ou nulos - 
#valores nulos sao considerados negativos tendo em vista o custo de oportunidade do ativo livre de risco

  retornos_positivos[i] = df_positivo
  retornos_negativos[i] = df_negativo

retornos_positivos = retornos_positivos.count() #conta o numero de elementos
retornos_negativos = retornos_negativos.count() 
todos_retornos = retornos_positivos+retornos_negativos #conta o numero total de retornos

valores_retornos_positivos = retornos_positivos/todos_retornos #%de retornos positivos
valores_retornos_negativos = retornos_negativos/todos_retornos #% de retornos negativos 

valores_pizza = pd.DataFrame() #primeira mente tinha feito em pizza, por isso o nome, após optei por barra
valores_pizza['positivos'] = valores_retornos_positivos
valores_pizza['negativos'] = valores_retornos_negativos


fig = go.Figure() #cria gráfico

fig.add_trace(go.Bar(x=valores_pizza.sort_values(by='positivos',ascending=False).index, 
                     y=valores_pizza.sort_values(by='positivos',ascending=False)['positivos'],
                     name='Positivos')) #positivos


fig.add_trace(go.Bar(x=valores_pizza.sort_values(by='positivos',ascending=False).index, 
                     y=valores_pizza.sort_values(by='positivos',ascending=False)['negativos'],
                     name='Negativos')) #negativos


fig.update_layout(
    height=600, 
    width=1300,
    barmode='relative',
    title = 'Porcentagem de retornos trimestrais',
    template="plotly_white",
    legend_title_text='Comparação entre:',
    legend_orientation="h",
    legend=dict(x=0, y=-.2),
    font=dict(
        family="Times New Roman",
        size=14,
        color="DimGray"
    ) #padroniza layout
)
fig.update_yaxes(title_text="Porcentagem (%)",
                  titlefont_size = 16,
                  tickfont_size=14,
                  nticks = 20,
                  color= 'DimGray'
                   ) #padroniza layout

fig.update_xaxes(title_text="Ativo ",
                  titlefont_size = 16,
                  tickfont_size=14,
                  nticks = 20,
                  color= 'DimGray'
                   ) #padroniza layout

fig.show()

fig.write_image(caminho+"/porcentagem_retorno_positivo_versus_negativos.png") #local do salvamento

## Evolução da rentabilidade dos ativos



In [446]:
fig = make_subplots(rows=2 ,
                    cols=1,
                    subplot_titles=('Evolução da rentabilidade do ativo','Drawdown do ativo'))
c=0
for i in informacoes_fundos['Acumulada']['VL_QUOTA'].columns:
  fig.add_trace(go.Scatter(y=informacoes_fundos['Acumulada']['VL_QUOTA'][i],
                           x=informacoes_fundos['Acumulada']['VL_QUOTA'][i].index, 
                           mode="lines+markers", marker=dict(size=5),
                           name=i,
                           legendgroup=i,
                           line=dict(color=cores_em_uso[c])),row=1,col=1)
  c+=1

c=0
for i in informacoes_fundos['Acumulada']['VL_QUOTA'].columns:
  fig.add_trace(go.Scatter(y=informacoes_fundos['Acumulada']['VL_QUOTA'][i].ewm(span=66, adjust=False).mean().dropna(),
                           x=informacoes_fundos['Acumulada']['VL_QUOTA'][i].ewm(span=66, adjust=False).mean().dropna().index, 
                           mode="lines", marker=dict(size=4),
                           name=i,
                           legendgroup=i,
                           showlegend=False,
                           line=dict(color=cores_em_uso[c])),row=1,col=1)
  c+=1

c=0
for i in informacoes_fundos['Acumulada']['VL_QUOTA'].columns:
  fig.add_trace(go.Scatter(y=((1 - informacoes_fundos['Acumulada']['VL_QUOTA'][i].div(informacoes_fundos['Acumulada']['VL_QUOTA'][i].cummax()))*-1).dropna(),
                           x=((1 - informacoes_fundos['Acumulada']['VL_QUOTA'][i].div(informacoes_fundos['Acumulada']['VL_QUOTA'][i].cummax()))*-1).dropna().index, 
                           mode="lines", marker=dict(size=5),
                           name=i,
                           fill='tozeroy',
                           legendgroup=i,
                           showlegend=False,
                           line=dict(color=cores_em_uso[c])),row=2,col=1)
  c+=1


fig.update_layout(
    height=700, 
    width=1300,
    title = 'Evolução acumulada dos ativos',
    template="plotly_white",
    legend_title_text='Comparação entre:',
    legend_orientation="h",
    legend=dict(x=0, y=-.2),
    font=dict(
        family="Times New Roman",
        size=14,
        color="DimGray"
    )
)


fig.update_yaxes(title_text="Rentabilidade (%)",
                  titlefont_size = 16,
                  tickfont_size=14,
                  nticks = 15,
                  color= 'DimGray'
                   ) 

fig.update_xaxes(title_text="Data ",
                  titlefont_size = 16,
                  tickfont_size=14,
                  nticks = 50,
                  color= 'DimGray'
                   ) 

fig.show()

fig.write_image(caminho+"/evolução_dos_ativos.png")

fig = go.Figure()

n=0
for i in informacoes_fundos['Acumulada']['VL_QUOTA'].columns:
  fig.add_trace(go.Indicator(
      value = informacoes_fundos['Acumulada']['VL_QUOTA'][i].ewm(span=66, adjust=False).mean().dropna().iloc[-1],
      delta = {'reference': informacoes_fundos['Acumulada']['VL_QUOTA'][i].ewm(span=66, adjust=False).mean().dropna().iloc[-22]},
      gauge = {
          'axis': {'visible': True}},
      domain = {'row': 0, 'column': n}, title=i,))
  n+=1
  fig.update_layout(
      grid = {'rows': 1, 'columns': len(informacoes_fundos['Acumulada']['VL_QUOTA'].columns), 'pattern': "independent"},
      template = {'data' : {'indicator': [{
          'mode' : "number+delta+gauge"}]
                           }},height=300, width=1300, title='Comparação entre trimestres (Atual - Mês passado)')
fig.show() #comparação entre o o trimestre atual e o trimestre do mês passado


## Frequência do retorno trimestral

In [ ]:
hist_data = []
for i in informacoes_fundos['Diaria']['VL_QUOTA'].columns:
  x1 = ((informacoes_fundos['Diaria']['VL_QUOTA'][i]+1).rolling(66).apply(np.prod)-1).dropna()
  hist_data.append(x1)
 
 
group_labels = informacoes_fundos['Diaria']['VL_QUOTA'].columns 
# Create distplot with custom bin_size
fig = ff.create_distplot(hist_data, group_labels, bin_size=.005, colors = cores_em_uso)

fig.update_layout(
    height=600, 
    width=1300,
    title = 'Frequência dos retornos trimestrais',
    template="plotly_white",
    legend_title_text='Comparação entre:',
    legend_orientation="h",
    xaxis_title='Data',
    yaxis_title="Volume",
    legend=dict(x=0, y=-.2),
    font=dict(
        family="Times New Roman",
        size=14,
        color="DimGray"
    )
)

fig.update_yaxes( titlefont_size = 16,
                  tickfont_size=14,
                  nticks = 20,
                  color= 'DimGray'
                   ) 

fig.update_xaxes( titlefont_size = 16,
                  tickfont_size=14,
                  nticks = 50,
                  color= 'DimGray'
                   ) 

fig.show()

In [ ]:
n=0
fig  =  make_subplots ( rows = 1 ,  cols = len(informacoes_fundos['Diaria']['VL_QUOTA'].columns) ,  shared_yaxes = True )
c=1
for i in informacoes_fundos['Diaria']['VL_QUOTA'].columns:
  fig.add_trace(go.Violin(y=((informacoes_fundos['Diaria']['VL_QUOTA'][i]+1).rolling(66).apply(np.prod)-1).dropna(), box_visible=True, line_color='black',
                                 meanline_visible=True, fillcolor=cores_em_uso[n], opacity=0.6, name=i,
                                x0=i,), row=1, col=c)
  c=c+1

  fig.update_layout(yaxis_zeroline=False)
  fig.update_layout(
    height=600, 
    width=1300,
    template="plotly_white",
    title='Concentração dos retornos',
    legend_title_text='Comparação entre:',
    legend_orientation="h",
    legend=dict(x=0, y=-.2),
    yaxis_title="Rentabilidade (%)",
    font=dict(
        family="Times New Roman",
        size=14,
        color="DimGray"
    )
)
  n=n+1


fig.update_yaxes( titlefont_size = 16,
                  tickfont_size=14,
                  nticks = 20,
                  color= 'DimGray'
                   ) 



fig.show()  


#Retorno Esperado

In [ ]:
amostra_aleatoria = pd.DataFrame() #armazenará a amostra 
for y in fundos_cnpj:
  valores = [] #lista de valores armazenados
  i=0 #variável controle 
  while i <100:#0000:
    aux = (informacoes_fundos['Diaria']['VL_QUOTA'][y].dropna().sample(66))
    aux = (((aux+1).rolling(66).apply(np.prod))-1).dropna().to_list()[0]
    valores.append(aux)
    i=i+1
  amostra_aleatoria[y] = valores
amostra_aleatoria.describe()

,37.306.536/0001-40
count,100.000000
mean,0.115695
std,0.081116
min,-0.066365
25%,0.068933
50%,0.114492
75%,0.167714
max,0.301628


In [367]:
for i in amostra_aleatoria.columns:  
  x = amostra_aleatoria[i].rolling(66).mean()

  fig = go.Figure(data=[go.Histogram(x=x)]) 
# Create distplot with custom bin_size
  fig.update_layout(
      title = 'Frequencia do retorno Trimestral do ativo: '+str(i),
      height=500, 
      width=1300,
      template="plotly_white",
      legend_title_text='Comparação entre:',
      legend_orientation="h",
      legend=dict(x=0, y=-.2),
      xaxis_title='Data',
      yaxis_title="Volume",
        
      font=dict(
          family="Times New Roman",
          size=14,
          color="DimGray"
      )

  )

  fig.update_yaxes( titlefont_size = 16,
                  tickfont_size=14,
                  nticks = 20,
                  color= 'DimGray'
                   ) 

  fig.update_xaxes( titlefont_size = 16,
                  tickfont_size=14,
                  nticks = 50,
                  color= 'DimGray'
                   ) 
  fig.show()


  maximos_minimos = pd.DataFrame()
for i in amostra_aleatoria.columns:
  df_vez = amostra_aleatoria.sort_values(by =i ,ascending=False).copy()
  ret_min = df_vez.iloc[int(len(df_vez)*.1):int(len(df_vez)*.9)].median()[i] - (2* df_vez.iloc[int(len(df_vez)*.1):int(len(df_vez)*.9)].std()[i])
  ret_max = df_vez.iloc[int(len(df_vez)*.1):int(len(df_vez)*.9)].median()[i] + (2* df_vez.iloc[int(len(df_vez)*.1):int(len(df_vez)*.9)].std()[i])
  maximos_minimos[i] = [ret_min,ret_max]
maximos_minimos.index = ['Mínimo esperado','Máximo esperado']
maximos_minimos

,37.306.536/0001-40
Mínimo esperado,0.010823
Máximo esperado,0.218161
